# Time Series Prediction with RNNs

The following example shows an adapted exercise from Aurélien Geron from his chapter on Recurrent Neural Networks [Hands-On Machine Learning with Scikit-Learn and TensorFlow]

In [1]:
import tensorflow as tf
import numpy as np
np.random.seed = 42

Functions to create a time series and train batches from it:

In [2]:
t_min, t_max = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

In [3]:
t = np.linspace(t_min, t_max, int((t_max - t_min) / resolution))

n_steps = 20
t_instance = np.linspace(12.2, 12.2 + resolution * (n_steps + 1), n_steps + 1)

Let's inspect the batches:

In [4]:
X_batch, y_batch = next_batch(1, n_steps)

In [5]:
np.c_[X_batch[0], y_batch[0]]

array([[ 4.59270276,  4.04624139],
       [ 4.04624139,  2.98698284],
       [ 2.98698284,  1.51059741],
       [ 1.51059741, -0.18979185],
       [-0.18979185, -1.86923196],
       [-1.86923196, -3.28920699],
       [-3.28920699, -4.27430027],
       [-4.27430027, -4.75339651],
       [-4.75339651, -4.77535366],
       [-4.77535366, -4.49532547],
       [-4.49532547, -4.13510097],
       [-4.13510097, -3.92718863],
       [-3.92718863, -4.05635237],
       [-4.05635237, -4.61292968],
       [-4.61292968, -5.56937675],
       [-5.56937675, -6.78579773],
       [-6.78579773, -8.04311778],
       [-8.04311778, -9.09579011],
       [-9.09579011, -9.73114324],
       [-9.73114324, -9.82084735]])

In [6]:
n_steps = 20
n_inputs = 1
n_neurons = 10
n_outputs = 1
n_epochs = 100
batch_size = 50
n_batches = int((30/0.1)/batch_size)
learning_rate = 0.1

## Graph Definition

#### Placeholder Definition

In [7]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

#### Inference Model with implicit Weights and Biases

In [8]:
basic_cell = tf.contrib.rnn.LSTMCell(num_units=n_neurons, activation=tf.nn.relu)
cell = tf.contrib.rnn.OutputProjectionWrapper(basic_cell, output_size=n_outputs)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

#### Loss, Optimization and Training Operation

In [9]:
loss = tf.reduce_mean(tf.square(outputs - y))  # MSE
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

## Graph Execution

In [10]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    for e in range(n_epochs):
        for b in range(n_batches):
            X_batch, y_batch = next_batch(batch_size, n_steps)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            it = e*n_batches*batch_size+b*batch_size
            if it % 2000 == 0:
                mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
                print(str(it), "\tMSE:", mse)

0 	MSE: 10.7633
2000 	MSE: 0.24237
4000 	MSE: 0.170618
6000 	MSE: 0.143206
8000 	MSE: 0.164518
10000 	MSE: 0.151998
12000 	MSE: 0.133611
14000 	MSE: 0.152168
16000 	MSE: 0.11749
18000 	MSE: 0.110844
20000 	MSE: 0.111466
22000 	MSE: 0.109123
24000 	MSE: 0.105656
26000 	MSE: 0.0969823
28000 	MSE: 0.0813314
